In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import keras

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

In [2]:
len(os.listdir("C:/Users/lenovo/Pictures/Disease_dataset/train"))

4

In [3]:
train_datagen = ImageDataGenerator(zoom_range= 0.5, shear_range= 0.3, 
                                   horizontal_flip= True,
                                   preprocessing_function= preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function= preprocess_input)


In [4]:
train = train_datagen.flow_from_directory(directory= "C:/Users/lenovo/Pictures/Disease_dataset/train", 
                                          target_size=(256,256), 
                                          batch_size= 4)

val = val_datagen.flow_from_directory(directory= "C:/Users/lenovo/Pictures/Disease_dataset/valid", 
                                          target_size=(256,256), 
                                          batch_size= 4)

Found 149 images belonging to 4 classes.
Found 150 images belonging to 4 classes.


In [9]:
t_image, label = train.next()

In [22]:
def plotImage(img_arr, label):
  for im, l in zip(img_arr, label):
    plt.figure(figsize=(5,5))
    plt.imshow(im)
    plt.show()

In [ ]:
plotImage(t_image[:3], label[:3])

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [10]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19

import keras

In [11]:
base_model = VGG19(input_shape=(256,256,3), include_top= False)

In [12]:
for layer in base_model.layers:
  layer.trainable = False

In [13]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [14]:
X = Flatten()(base_model.output)

X = Dense(units= 4, activation='softmax')(X)


#Final Model
model = Model(base_model.input, X)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [16]:
model.compile(optimizer= 'adam', loss= keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [17]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

#Early stopping
es = EarlyStopping(monitor= 'val_accuracy', min_delta= 0.01, patience= 3, verbose= 1)

#Model check point

mc = ModelCheckpoint(filepath= 'best_model.h5', 
                     monitor= 'val_accuracy', 
                     min_delta= 0.01, patience= 3, 
                     verbose= 1, 
                     save_best_only= True)

cb = [es, mc]

In [ ]:
his = model.fit_generator(train, 
                          steps_per_epoch= 18,
                          epochs= 80, 
                          verbose= 1, 
                          callbacks= cb, 
                          validation_data= val, 
                          validation_steps= 16)

D:\anaconda\envs\project_dl\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/80
18/18 [==============================] - 442s 25s/step - loss: 26.6712 - accuracy: 0.3333 - val_loss: 29.2789 - val_accuracy: 0.5156

Epoch 00001: val_accuracy improved from -inf to 0.51562, saving model to best_model.h5
Epoch 2/80
18/18 [==============================] - 439s 25s/step - loss: 29.9441 - accuracy: 0.4028 - val_loss: 31.5682 - val_accuracy: 0.4219

Epoch 00002: val_accuracy did not improve from 0.51562
Epoch 3/80
18/18 [==============================] - 445s 25s/step - loss: 19.3789 - accuracy: 0.5000 - val_loss: 22.6689 - val_accuracy: 0.5781

Epoch 00003: val_accuracy improved from 0.51562 to 0.57812, saving model to best_model.h5
Epoch 4/80
 2/18 [==>...........................] - ETA: 4:01 - loss: 11.2232 - accuracy: 0.7500

In [ ]:
h = his.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c = 'red')
plt.title("acc vs V-acc")
plt.show()

In [5]:
from keras.models import load_model

model = load_model("C:/Users/lenovo/Pictures/Disease_dataset/best_model.h5")

#acc = model.evaluate_generator(val)[1]
#print(f"The accuracy of the model is = {acc*100} %")

In [14]:
def prediction(path):
    print("Please wait...")
    img = load_img(path, target_size= (256,256))
    i = img_to_array(img)
    im = preprocess_input(i)
    img = np.expand_dims(im, axis= 0)
    print(model.predict(img))
    pred = np.argmax(model.predict(img))
    print(f"result of your file is {pred}")

In [17]:
path = "C:/Users/lenovo/Pictures/Disease_dataset/test/CORDANA_02.jpg"
prediction(path)

Please wait...
[[0.0000000e+00 1.0000000e+00 3.0766705e-30 3.3412977e-16]]
result of your file is 1


In [8]:
train.class_indices

{'BUNCHY_TOP': 0, 'CORDANA': 1, 'PANAMA': 2, 'SIGATOKA': 3}